In [2]:
from datasets import Dataset
import pandas as pd
from fontTools.ttLib.tables.ttProgram import instructions
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, GenerationConfig

import torch
from peft import LoraConfig, TaskType, get_peft_model

from tqdm.notebook import tqdm
import csv
import os
from peft import PeftModel
# from policy_crawling.datacleaning.test import tokenizer

In [2]:
# 将Json文件转换为CSV文件
df = pd.read_json('qwen微调/微调数据_改.json')
ds = Dataset.from_pandas(df)


In [3]:
ds[:3]

{'instruction': ['你是一名实体提取、意图识别和分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。\n\n任务：\n1-分析用户输入问题的语义信息和语法结构。\n2-根据分析结果，将问题中的停用词、无意图词、语气词和符号剔除。\n3-检查是否存在剔除遗漏，是否存在错误剔除，并进行矫正。\n4-将保留的文本进行实体、关键词、意图词、主语词、设备词识别。\n5-识别出的词以逗号分隔。\n6-根据分析结果、实体提取结果和给出的意图分类标签进行问题的意图分类。\n7-根据输出格式输出。\n\n意图标签：商品数据，招标中标信息，政策文件信息，日常问题。\n\n请根据分类结果与要求的输出格式对比，存在格式问题则进行矫正后输出。\n\n输出格式：\n实体识别结果：实体，实体\n意图分类：意图标签\n\n请注意输出结果严格按照输出格式进行，不需要输出分析过程。不要输出实体结果。\n\n问题：医用DR设备都有哪些品牌？',
  '你是一名实体提取、意图识别和分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。\n\n任务：\n1-分析用户输入问题的语义信息和语法结构。\n2-根据分析结果，将问题中的停用词、无意图词、语气词和符号剔除。\n3-检查是否存在剔除遗漏，是否存在错误剔除，并进行矫正。\n4-将保留的文本进行实体、关键词、意图词、主语词、设备词识别。\n5-识别出的词以逗号分隔。\n6-根据分析结果、实体提取结果和给出的意图分类标签进行问题的意图分类。\n7-根据输出格式输出。\n\n意图标签：商品数据，招标中标信息，政策文件信息，日常问题。\n\n请根据分类结果与要求的输出格式对比，存在格式问题则进行矫正后输出。\n\n输出格式：\n实体识别结果：实体，实体\n意图分类：意图标签\n\n请注意输出结果严格按照输出格式进行，不需要输出分析过程。不要输出实体结果。\n\n问题：动态三维脊柱姿态测量分析装置北京的供应商有哪些品牌？',
  '你是一名实体提取、意图识别和分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。\n\n任务：\n1-分析用户输入问题的语义信息和语法结构。\n2-根据分析结果，将问题中的停用词、无意图词、语气词和符号剔除。\n3-检查是否存在剔除遗漏，是否存在错误剔除，并进行矫正。\n4-将保留的文本进行实体、关键词、意

In [4]:
# 下载qwen2.5-3b模型
# from modelscope import snapshot_download
# model_dir = snapshot_download("Qwen/Qwen2.5-3B-Instruct", revision="master", cache_dir='./models')

# 加载tokenizer

In [5]:
# 加载分词器
model_dir = 'autodl-tmp/Qwen/Qwen2.5-3B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False, trust_remote_code=True)
tokenizer

Qwen2Tokenizer(name_or_path='autodl-tmp/Qwen/Qwen2.5-3B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_wor

In [6]:
# 构建数据转编码函数
def process_func(example):
    MAX_LENGTH = 1800
    instruction = tokenizer(f"<|im_start|>system\n你是一名实体提取、意图识别和分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。<|im_end|>\n<|im_start|>user\n{example['instruction']+example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction['input_ids'] + response['input_ids'] + [tokenizer.pad_token_id]
    attention_mask = instruction['attention_mask'] + response['attention_mask'] + [1]
    labels = [-100] * len(instruction['input_ids']) + response['input_ids'] + [tokenizer.pad_token_id]

    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }


# 数据格式化

In [7]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/437 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 437
})

In [8]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n你是一名实体提取、意图识别和分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。<|im_end|>\n<|im_start|>user\n你是一名实体提取、意图识别和分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。\n\n任务：\n1-分析用户输入问题的语义信息和语法结构。\n2-根据分析结果，将问题中的停用词、无意图词、语气词和符号剔除。\n3-检查是否存在剔除遗漏，是否存在错误剔除，并进行矫正。\n4-将保留的文本进行实体、关键词、意图词、主语词、设备词识别。\n5-识别出的词以逗号分隔。\n6-根据分析结果、实体提取结果和给出的意图分类标签进行问题的意图分类。\n7-根据输出格式输出。\n\n意图标签：商品数据，招标中标信息，政策文件信息，日常问题。\n\n请根据分类结果与要求的输出格式对比，存在格式问题则进行矫正后输出。\n\n输出格式：\n实体识别结果：实体，实体\n意图分类：意图标签\n\n请注意输出结果严格按照输出格式进行，不需要输出分析过程。不要输出实体结果。\n\n问题：医用DR设备都有哪些品牌？<|im_end|>\n<|im_start|>assistant\n实体识别结果：医用DR设备，品牌\n意图分类：商品数据<|endoftext|>'

In [9]:
# 需要过滤掉-100的位置既是输入的文本信息
tokenizer.decode(list(filter(lambda x:x!=-100, tokenized_id[0]['labels'])))

'实体识别结果：医用DR设备，品牌\n意图分类：商品数据<|endoftext|>'

# 加载半精度模型

In [10]:

model = AutoModelForCausalLM.from_pretrained(model_dir,
                                             trust_remote_code=True,
                                             # device_map='auto',
                                             torch_dtype=torch.bfloat16)
model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm):

In [11]:
model.enable_input_require_grads()  # 开启梯度检查点时，要执行该方法

In [12]:
# model.dtype

# 定义LoraConfig

In [3]:

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    # 需要训练的模型层的名字，主要就是attention部分的层，
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False,   # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alpha 控制了 LoRA 变换的缩放因子
    lora_dropout=0.1,  # Lora dropout
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'down_proj', 'v_proj', 'q_proj', 'k_proj', 'up_proj', 'o_proj', 'gate_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [14]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='autodl-tmp/Qwen/Qwen2.5-3B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'gate_proj', 'q_proj', 'down_proj', 'v_proj', 'k_proj', 'o_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [15]:
model.print_trainable_parameters()

trainable params: 14,966,784 || all params: 3,100,905,472 || trainable%: 0.4827


# 配置训练参数

In [45]:
args = TrainingArguments(
    output_dir='./autodl-tmp/output/Qwen3b_instruct_lora_3',
    # batch_size
    per_device_train_batch_size=4,
    # 梯度累加，如果你的显存比较小，那可以把 batch_size 设置小一点，梯度累加增大一些
    gradient_accumulation_steps=4,
    # 多少步，输出一次log
    logging_steps=20,
    # 顾名思义 epoch
    num_train_epochs=20,
    save_steps=20,
    learning_rate=1e-4,
    save_on_each_node=True,
    # 梯度检查，这个一旦开启，模型就必须执行model.enable_input_require_grads()
    gradient_checkpointing=True,
    # save_safetensors=False
)

In [46]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

In [47]:
# 禁用内存高效的张量收缩算法,这个函数通常用于改善大规模矩阵乘法操作的内存使用效率，特别是在使用 GPU 时。
# torch.backends.cuda.enable_mem_efficient_sdp(False)


In [48]:
# 开始训练
trainer.train()

Step,Training Loss
20,0.013300
40,0.002600
60,0.000600
80,0.002200
100,0.001000
120,0.003800
140,0.000500
160,0.000200
180,0.001200
200,0.000500


TrainOutput(global_step=540, training_loss=0.0011012348680567272, metrics={'train_runtime': 1635.5435, 'train_samples_per_second': 5.344, 'train_steps_per_second': 0.33, 'total_flos': 4.324172608246579e+16, 'train_loss': 0.0011012348680567272, 'epoch': 19.636363636363637})

# 加载 lora 权重推理
## 合并加载模型

In [4]:
device = torch.device('cuda')
model_path = 'autodl-tmp/Qwen/Qwen2.5-3B-Instruct'
lora_path = 'autodl-tmp/output/Qwen3b_instruct_lora_3/checkpoint-540'
tokenizer_ = AutoTokenizer.from_pretrained(model_path)
model_init = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16).eval().to(device)
model_lora = PeftModel.from_pretrained(model_init, model_id=lora_path, config=config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
datas = pd.read_excel('意图识别数据集-eval(1).xlsx') 
questions = datas['问题']
labels = datas['问题分类']
# bodys = datas['提取的实体']
# sublabel = set(labels.values.tolist())
file = '实体提取_意图分类结果_lora微调_1126_4.csv'
for question, label in tqdm(zip(questions, labels), ncols=len(questions)) :
    template = f"""
            你是一名实体提取、意图识别和分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。
            
            任务：
            1-分析输入问题的文本和语法结构。
            2-根据分析结果，将问题中的停用词、无意图词、语气词和符号剔除。
            3-检查是否存在剔除遗漏，是否存在错误剔除，并进行矫正。
            4-将保留的文本进行实体、关键词、意图词、主语词、设备词识别。
            5-识别出的词以逗号分隔。
            6-根据分析结果、实体提取结果和给出的意图分类标签进行问题的意图分类。
            7-根据输出格式输出。
    
            意图标签：商品数据，招标中标信息，政策文件信息，日常问题。
            
            请根据分类结果与要求的输出格式对比，存在格式问题则进行矫正后输出。
                
            输出格式：
                实体识别结果：实体，实体
                意图分类：意图标签
    
            请注意输出结果严格按照输出格式进行，不需要输出分析过程。不要输出实体结果
    
            问题：{question}"""
    messages = [
                    # {'role':'system', 'content': template},
                    {'role':'user', 'content': template}
                    ]
        # print('打印messages内容：', messages)
        # print('打印messages类型：', type(messages))
    text = tokenizer_.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer_([text], return_tensors='pt').to(device)
    generated_ids = model_lora.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer_.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print('question:', question)
    print('response', response)
    body_ = response.split('\n')[0].split('实体识别结果：')[-1].replace(' ','')
    emition = response.split('意图分类：')[-1].replace(' ','')
    data_ = [question, body_, label, emition]
    with open(file, 'a', newline='', encoding='utf-8') as f:
        fw = csv.writer(f)
        if not os.path.getsize(file):
            header = ['问题', '提取的实体', '问题分类', '意图分类']
            fw.writerow(header)
        fw.writerow(data_)

0it [00:00, ?it/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


question: 【综合采购】上海市公共资源交易中心综合采购专家库管理实施规则(试行)的发布时间是什么？
response 实体识别结果：综合采购，上海市公共资源交易中心综合采购专家库管理实施规则(试行)，发布时间
意图分类：政策文件信息
question: 【综合采购】上海市公共资源交易中心综合采购专家库管理实施规则(试行)的信息来源是哪里？
response 实体识别结果：综合采购，上海市公共资源交易中心，综合采购专家库管理实施规则(试行)，信息来源
意图分类：政策文件信息
question: 在招投标工作期间，工作人员有哪些行为是被禁止的？
response 实体识别结果：工作人员，行为，禁止
意图分类：政策文件信息
question: 保险公司股权管理办法(2018)发布于那一年？
response 实体识别结果：保险公司股权管理办法(2018)，发布，年份
意图分类：政策文件信息
question: 怎么和陌生人开启话题？
response 实体识别结果：陌生人，开启话题
意图分类：日常问题
question: 如果会员违反了配额最大持有量限制制度，交易所会采取哪些措施？
response 实体识别结果：会员，配额最大持有量限制制度，交易所，措施
意图分类：政策文件信息
question: 《财政部关于促进政府采购公平竞争优化营商环境的通知》的信息来源是哪里？
response 实体识别结果：《财政部关于促进政府采购公平竞争优化营商环境的通知》，信息来源
意图分类：政策文件信息
question: 【产权交易】上海联合产权交易所有限公司企业非公开协议增资操作流程（沪联产交〔2019〕77号）的信息来源是哪里？
response 实体识别结果：产权交易，上海联合产权交易所有限公司，企业非公开协议增资操作流程，沪联产交〔2019〕77号，信息来源
意图分类：政策文件信息
question: 医院一代测序仪属于什么类型的商品？
response 实体识别结果：医院一代测序仪，类型，商品
意图分类：商品数据
question: 今天天气怎么样？
response 实体识别结果：天气
意图分类：日常问题
question: 你能给我推荐一本好书吗？
response 实体识别结果：好书
意图分类：日常问题
question: 流式细胞仪项目的中标金额最低是多少？
